In [8]:
# load the data from API 
import pandas as pd
from sodapy import Socrata
from tqdm import tqdm
import time

In [9]:
# Loading 311 data for New York City 
client = Socrata("data.cityofnewyork.us", None)
DATASET_ID = "erm2-nwe9"
CHUNK_SIZE = 10000         
MAX_RECORDS = 100000       
OUTPUT_FILE = "nyc_311_sample.csv"

all_results = []

for offset in tqdm(range(0, MAX_RECORDS, CHUNK_SIZE)):
    try:
        results = client.get(
            DATASET_ID,
            limit=CHUNK_SIZE,
            offset=offset,
            order="created_date DESC"  # newest first
        )
        if not results:
            break
        df_chunk = pd.DataFrame.from_records(results)
        all_results.append(df_chunk)

        time.sleep(0.2)

    except Exception as e:
        print(f"Error at offset {offset}: {e}")
        break

if all_results:
    full_df = pd.concat(all_results, ignore_index=True)
    full_df.to_csv(OUTPUT_FILE, index=False)
    print(f"Saved {len(full_df)} rows to {OUTPUT_FILE}")
else:
    print("No data retrieved.")

100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


✅ Saved 100000 rows to nyc_311_sample.csv


In [16]:
# Load data for census

#dataset3• (API)
borough_map = {
    'MANHATTAN': '061', #•FIPS• for New York County
    'BROOKLYN': '047',
    'QUEENS': '081',
    'BRONX': '005',
    'STATEN ISLAND': '085'# Richmond • County
}
median_income_data = []
for boro, fips in borough_map.items ():
    # state36 = NY
    #APT •calli
    url = f"https://api.census.gov/data/2023/acs/acs1?get=B19013_001E&for=county:{fips}&in=state:36&key=b615b96b9abca0f75f96cf64ef5bbe3cfcf19bfe"
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
        median_income_data.append((boro, data [1][0])) #•data [1] [0] •= median income, • from documentation

df_income = pd.DataFrame(median_income_data, columns=['Work Location Borough', 'median_income'])

df_final = pd-merge(
    df_merged,
    df_income,
    how='left', on='Work-Location Borough'
)

NameError: name 'requests' is not defined